In [9]:
import pandas as pd
import numpy as np
aid_data = pd.read_csv('a2019.csv',encoding='UTF-8',skipinitialspace=True)
aid_data1 = aid_data
aid_data = pd.read_csv('a2019.csv',encoding='UTF-8',skipinitialspace=True)

aid_data1 = aid_data

aid_data1.columns= aid_data1.columns.str.strip()

# aid_data1 = aid_data1.dropna(subset=['c5'],how='all')

aid_data1 = aid_data1.drop(aid_data1.columns[[-1,-2]],axis=1)

fil = aid_data1['c5'].str.contains(r'^[a-zA-Z]')

aid_data1 = aid_data1[~fil]

aid_data1.reset_index(drop=True,inplace=True)

aid_data1['c9'] = pd.to_datetime(aid_data1['c9'].astype(str), format='%Y%m%d',errors='coerce')
aid_data1['c205'] = pd.to_datetime(aid_data1['c205'].astype(str), format='%Y%m%d',errors='coerce')
aid_data1['c206'] = pd.to_datetime(aid_data1['c206'].astype(str), format='%Y%m%d',errors='coerce')
aid_data1['c207'] = pd.to_datetime(aid_data1['c207'].astype(str), format='%Y%m%d',errors='coerce')
aid_data1['c208'] = pd.to_datetime(aid_data1['c208'].astype(str), format='%Y%m%d',errors='coerce')
aid_data1['c210'] = pd.to_datetime(aid_data1['c210'].astype(str), format='%Y%m%d',errors='coerce')
aid_data1['c229'] = pd.to_datetime(aid_data1['c229'].astype(str), format='%Y%m%d',errors='coerce')
aid_data1['c230'] = pd.to_datetime(aid_data1['c230'].astype(str), format='%Y%m%d',errors='coerce')
aid_data1['c10'] = pd.to_datetime(aid_data1['c10'].astype(str),format= '%H%M.0',errors='coerce').dt.strftime('%H:%M')

aid_data1[aid_data1.columns[79:95]] = aid_data1[aid_data1.columns[79:95]].fillna(0)

for col in aid_data1.columns[80:93].values:
    aid_data1[col] = aid_data1[col].astype('int64')

aid_data1[['c149','c151','c145']] = aid_data1[['c149','c151','c145']].fillna(0).astype(int).astype(str).replace('0','NaN')

aid_data1.rename(columns={'c9': 'Date_Of_Occurrence', 'c22': 'N_Number'}, inplace=True)

aid_data1 = aid_data1.add_suffix('_aid').rename(columns={'Report_Type_aid': 'Report_Type', 'Date_Of_Occurrence_aid': 'Date_Of_Occurrence','N_Number_aid':'N_Number'})

aid_data1.insert(0,'Report_Type','AID')
    
# aid_data1[aid_data1.columns[79:95]]

In [10]:
aid_data1_a = aid_data1[['Report_Type','c5_aid']]

In [11]:
aid_data1_a.shape

(2391, 2)

In [12]:
import more_itertools as mit
e_aid_data = pd.read_csv('e2019(possible).csv',encoding='UTF-8',skipinitialspace=True)
e_aid_data1= e_aid_data

e_aid_data1 = e_aid_data1.dropna(subset=['c5'],how='all')

e_aid_data1['remark'] = e_aid_data1[['remark','Unnamed: 2','Unnamed: 3','Unnamed: 4','Unnamed: 5','Unnamed: 6','Unnamed: 7','Unnamed: 8','Unnamed: 9']].replace('?','').fillna('').agg(' '.join, axis=1)
e_aid_data1 = e_aid_data1.drop(e_aid_data1.columns[[2,3,4,5,6,7,8,9]],axis=1)

e_aid_data1.reset_index(drop=True,inplace=True)

e_aid_data1['remark']= e_aid_data1['remark'].replace(r'^\s*$', np.nan, regex=True)

e_aid_data1['remark']= e_aid_data1['remark'].mask(pd.isnull, e_aid_data1['c5'])


fill = e_aid_data1['c5'].str.contains(r'^\b2019')

e_aid_data1['c5'][fill==False] = np.nan

q=[index for index, row in e_aid_data1.iterrows() if row.isnull().any()]

q=[list(group) for group in mit.consecutive_groups(q)]

for i in q:
    t=''
    s=i[0]
    for j in i:
        t+=(e_aid_data1.iloc[j]['remark'])
        t+=' '
    e_aid_data1.iloc[s-1]['remark']+=t
        
e_aid_data1 = e_aid_data1[pd.notnull(e_aid_data1['c5'])]

e_aid_data1.reset_index(drop=True,inplace=True)

e_aid_data1.rename(columns={'c5':'c5_aid','remark': 'Remarks'},inplace=True)

e_aid_data1['Remarks'] = e_aid_data1['Remarks'].str.replace('\^PRIVACY DATA OMITTED\^', '').str.strip().str.replace('\s+', ' ', regex=True)

# e_aid_data1['Remarks'] = e_aid_data1['Remarks'].str.replace('iaw srm dwg', '').str.strip().str.replace('\s+', ' ', regex=True)

# e_aid_data1 = e_aid_data1.add_suffix('_aid')

e_aid_data1.insert(0,'Report_Type','AID')

e_aid_data1
e_aid_data1.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


(2408, 3)

In [13]:
aid_data1_nlp = pd.merge(aid_data1_a, e_aid_data1, how="outer", on=["Report_Type", "c5_aid"])

aid_data1_nlp = aid_data1_nlp[['Report_Type','Remarks']]

aid_data1_nlp

,Report_Type,Remarks
0,AID,THE PILOT OBTAINED LOCAL AUTOMATED WEATHER REP...
1,AID,DEPARTED TRL FOR FLIGHT TO GLADEWATER. CLIMBED...
2,AID,"ON NOVEMBER 25, 2019, AT APPROXIMATELY 1130 EA..."
3,AID,THE PILOT/OWNER WAS INTENDING TO MAKE A SHORT ...
4,AID,A PORTION OF THE PROPELLER SEPARATED FROM AIRC...
...,...,...
2403,AID,"ON MONDAY, MAY 13, 2019 AT APPROXIMATELY 1221 ..."
2404,AID,"ON JUNE 1, 2019, ABOUT 2030 CENTRAL STANDARD T..."
2405,AID,"WHILE ON APPROACH TO LANDING, N72LB, A LINSTRA..."
2406,AID,PILOT FAILED TO MAINTAIN DIRECTIONAL CONTROL O...


In [ ]:
aid_data1_nlp.Remarks[aid_data1_nlp.Remarks.str.contains('(SMOKE)|(FUMES)|(FIRE)|(ODORS)|(SPARKE)')]

In [ ]:
aid_data1_nlp.Remarks[aid_data1_nlp.Remarks.str.contains('(BIRD STRIKE)')]

In [16]:
sdr_data = pd.read_csv('sdr(2019_Occurred_Incidents).csv',encoding='UTF-8',skipinitialspace=True,dtype=str)
sdr_data1 = sdr_data

sdr_data1 = sdr_data1.dropna(subset=['c25'],how='all')

sdr_data1 = sdr_data1.drop(sdr_data1.columns[[-1,-2,-3,0]],axis=1)
sdr_data1.columns= sdr_data1.columns.str.strip()

sdr_data1['c10'] = pd.to_datetime(sdr_data1['c10'].astype(str), format='%Y%m%d',errors='coerce')
sdr_data1['c25'] = pd.to_datetime(sdr_data1['c25'].astype(str), format='%Y%m%d',errors='coerce')

sdr_data1['Remarks'] = sdr_data1[['c510a','c510b','c510c','c510d','c510e','c510f']].fillna('').agg(' '.join, axis=1)

sdr_data1['Remarks']= sdr_data1['Remarks'].str.strip().str.replace('\s+', ' ', regex=True)

sdr_data1.drop(['c510a', 'c510b','c510c','c510d','c510e','c510f'], axis=1, inplace=True)

sdr_data1.rename(columns={'c25': 'Date_Of_Occurrence', 'c390': 'N_Number'}, inplace=True)

sdr_data1 = sdr_data1.add_suffix('_sdr').rename(columns={'Report_Type_sdr': 'Report_Type', 'Date_Of_Occurrence_sdr': 'Date_Of_Occurrence','N_Number_sdr':'N_Number','Remarks_sdr':'Remarks'})

sdr_data1.insert(0,'Report_Type','SDR')


In [17]:
eon_data = pd.read_csv('EON-2019(Jan-Mar).csv',encoding='UTF-8',skipinitialspace=True)
eon_data1 = eon_data
eon_data1.columns= eon_data1.columns.str.strip()

eon_data1['Event Date'] = pd.to_datetime(eon_data1['Event Date'])
eon_data1['Event Time'] = eon_data1['Event Date'].dt.strftime('%H:%M')
eon_data1['Event Date'] = eon_data1['Event Date'].dt.strftime('%Y-%m-%d')

eon_data1['Event Date'] = pd.to_datetime(eon_data1['Event Date'])

cols = eon_data1.columns.tolist()
cols = cols[0:5]+cols[-1:]+cols[5:36]
eon_data1 = eon_data1[cols]

eon_data1['Message Whole'] = eon_data1[['MessageText.1','Unnamed: 30','Unnamed: 31','Unnamed: 32','Unnamed: 33','Unnamed: 34','Unnamed: 35']].fillna('').agg(' '.join, axis=1)

eon_data1['Message Whole'] = eon_data1['Message Whole'].str.strip().str.replace('\s+', ' ', regex=True)

eon_data1.rename(columns={'Message Whole': 'Remarks','Event Date': 'Date_Of_Occurrence'},inplace=True)

cols1 = eon_data1.columns.tolist()
cols1 = cols1[0:29] + cols1[-1:]

eon_data1 = eon_data1[cols1]

eon_data1['N_Number'] = eon_data1['Title'].str.extract(r'((?<=\|).+?(?=\|))')

eon_data1['N_Number'] = eon_data1['N_Number'].str.strip()

fil = eon_data1['N_Number'].str.contains('^N[0-9]')

eon_data1['N_Number'][fil==True] = eon_data1['N_Number'][fil==True].apply(lambda x : x[1:] if str(x).startswith("N") else x)

eon_data1.insert(0,'Report_Type','EON')

# eon_data1.drop(['Title'], axis=1, inplace=True)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/i

In [18]:
sdr_data1_nlp = sdr_data1[['Report_Type','Remarks']]

eon_data1_nlp = eon_data1[['Report_Type','Remarks']]

eon_data1_nlp.Remarks = eon_data1_nlp.Remarks.str.lstrip(',').str.strip()

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [19]:
sdr_data1_nlp.Remarks[sdr_data1_nlp.Remarks.str.contains('(BIRD STRIKE)')]

/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


4651     AIRCRAFT WAS GROUNDED: PILOT REPORTS BIRD INGE...
8405     AIRCRAFT WAS GROUNDED: MAINTENANCE REPORTS EVI...
8775     AIRCRAFT WAS NOT GROUNDED: AT APPROXIMATLEY 87...
16752    WE HAD A BIRD STRIKE AT 5000 FEET AND 250 KNOT...
22375    IMMEDIATELY AFTER TAKEOFF GOT A BURNING SMELL ...
25617    MDIS 917 57-50 0329 14MAY19 LGG OPEN U REF WRI...
28224    STRONG PROPANE ODOR AFT GALLEY IN CRUISE. APPL...
29175    ATB - BIRD STRIKE ON DEPARTURE. RIGHT SIDE OF ...
40679    BIRD STRIKE ON T/O ROLL ON NOSE CONE AN 2 RH W...
42516    LT FRONT WINDOW CRACKED WHILE ON APPROACH. DOE...
45220    DURING CLIMBOUT, STRONG FUMES IN THE CABIN AND...
56078    BIRD STRIKE RIGHT ENGINE(<P>PERFORMED&NBSP; #2...
56097    MOMENTARY WX FAIL AFTER BIRD STRIKE.(<P>ACCOMP...
58715    (CAN) CREW REPORTED A LOUD BANG AND VIBRATION ...
Name: Remarks, dtype: object

In [20]:
sdr_data1_nlp.Remarks[sdr_data1_nlp.Remarks.str.contains('(SMOKE)|(FUMES)|(FIRE)|(ODORS)|(SPARKE)')]

/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


3        RH ENGINE FIRE LOOP 1 DISPLAYED ON EICAS ON PR...
5        IN FLIGHT, AFT CABIN CREW DETAILED A BURNING S...
38       AS REPORTED BY THE PIC: WHILE IN FLIGHT NR 2 F...
86       FLIGHT# 0117, ELECTRICAL BURNING SMELL IN COCK...
146      FLIGHT# 0078, CAPTAIN SIDE HAD NO ABILITY TO C...
                               ...                        
60533    WHEEL WELL OVERHEAT LIGHT CAME ON WITHOUT AUDI...
60552    CORROSION ON THE L/H AND R/H FUSELAGE AT THE E...
60573    AIRCRAFT HAS HAD A HISTORY OF SOFT BRAKES, BRA...
60681    RETURN FLIGHT KCHO TO KFCI, PILOT ENCOUNTERED ...
60788    WE WERE CLIMBING SOMEWHERE IN THE LOW 20’S. WE...
Name: Remarks, Length: 1997, dtype: object

In [21]:
eon_data1_nlp.Remarks[eon_data1_nlp.Remarks.str.contains('(BIRD STRIKE)')]

/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


6       E145, AVP-ORD, ABORTED TAKEOFF AND RETURNED TO...
13      B737, SMFBIRD STRIKE ROLL OUT RWY 34L. TAXIED ...
24      MD88, ATL-MEM, REPORTED A BIRD STRIKE LANDING ...
41      B739, SEA-SJC, REPORTED AN BIRD STRIKE DURING ...
64      A332, REPORTED A POSSIBLE BIRD STRIKE DEPARTIN...
                              ...                        
9427    B772, IAD-EGLL, REPORTED A BIRD STRIKE AFTER D...
9433    E75L, PILOT REPORTED A BIRD STRIKE ON TOUCHDOW...
9440    B752, RST-MEM, REPORTED A BIRD STRIKE ON A 1 M...
9482    CRJ7, CLT-AGS, REPORTED A BIRD STRIKE ON LANDI...
9535    B738, ORD-MSY, PILOT REPORTED BIRD STRIKE WHIL...
Name: Remarks, Length: 375, dtype: object

In [22]:
eon_data1_nlp.Remarks[eon_data1_nlp.Remarks.str.contains('(SMOKE)|(FUMES)|(FIRE)|(ODORS)|(SPARKE)')]

/usr/local/lib/python3.7/dist-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


22      B737, WAS ON FINAL TO RWY34L, HE REPORTED A FI...
37      B737, STL-SAT, DECLARED AN EMERGENCY FOR ASYME...
62      A321, DFW-SAN, DECLARED AN EMERGENCY AND DIVER...
125     C172, LANDED RY17L WITH OUT INCIDENT AFTER DEC...
180     CRJ9, OKC-DTW, ABORTED TAKEOFF ON RUNWAY 35R D...
                              ...                        
9528    C172, REPORTED A ELECTRICAL FIRE AND SMOKE IN ...
9581    RV7, PRIOR TO DEPARTURE IN RUN-UP AREA, BRAKE ...
9592    C130, RAIDR02, A C130 TAXIED OUT TO RWY 25L AT...
9649    CJ3, REPORTED A POSSIBLE ENGINE FIRE WHILE STA...
9660    B772, EHAM-JFK, REPORTED AN ENGINE FIRE INDICA...
Name: Remarks, Length: 327, dtype: object

In [23]:
aid_sdr_eon_nlp = aid_data1_nlp.append([sdr_data1_nlp, eon_data1_nlp])

aid_sdr_eon_nlp.reset_index(drop=True,inplace=True)

aid_sdr_eon_nlp.columns = aid_sdr_eon_nlp.columns.str.strip().str.replace('\s+', ' ', regex=True)

aid_sdr_eon_nlp

,Report_Type,Remarks
0,AID,THE PILOT OBTAINED LOCAL AUTOMATED WEATHER REP...
1,AID,DEPARTED TRL FOR FLIGHT TO GLADEWATER. CLIMBED...
2,AID,"ON NOVEMBER 25, 2019, AT APPROXIMATELY 1130 EA..."
3,AID,THE PILOT/OWNER WAS INTENDING TO MAKE A SHORT ...
4,AID,A PORTION OF THE PROPELLER SEPARATED FROM AIRC...
...,...,...
73135,EON,"EVIATION REPORTED BY ORF WHEN PAT092, C560, FA..."
73136,EON,"EVIATION REPORTED BY ZMP WHEN N339MK, P28A/G, ..."
73137,EON,"EVIATION REPORTED BY ZAB WHEN OXF165, P28A, CL..."
73138,EON,"EVIATION REPORTED BY SCT WHEN SKW3264, CRJ7, T..."


In [24]:
import re
import unicodedata
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
import matplotlib.pyplot as plt
# add appropriate words that will be ignored in the analysis
# ADDITIONAL_STOPWORDS = ['covfefe']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [25]:
def basic_clean(text):
  """
  A simple function to clean up the data. All the words that
  are not designated as a stop word is then lemmatized after
  encoding and basic regex parsing are performed.
  """
  wnl = nltk.stem.WordNetLemmatizer()
  stopwords = nltk.corpus.stopwords.words('english')
  text = (unicodedata.normalize('NFKD', text)
    .encode('ascii', 'ignore')
    .decode('utf-8', 'ignore')
    .lower())
  words = re.sub(r'[^\w\s]', '', text).split()
  return [wnl.lemmatize(word) for word in words if word not in stopwords]

In [26]:
words_aid = basic_clean(''.join(str(aid_data1_nlp['Remarks'].tolist())))

In [27]:
words_sdr = basic_clean(''.join(str(sdr_data1['Remarks'].tolist())))

In [28]:
words_eon = basic_clean(''.join(str(eon_data1['Remarks'].tolist())))

In [29]:
words_eon = basic_clean(''.join(str(eon_data1['Remarks'].tolist())))

In [30]:
(pd.Series(nltk.ngrams(words_aid, 1)).value_counts())[:60]

(aircraft,)         6173
(pilot,)            4400
(runway,)           2811
(landing,)          2510
(gear,)             2017
(flight,)           1976
(left,)             1683
(airport,)          1570
(right,)            1527
(engine,)           1453
(damage,)           1365
(approximately,)    1315
(time,)             1116
(wing,)             1002
(stated,)            960
(nose,)              921
(ground,)            865
(2019,)              843
(fuel,)              811
(foot,)              785
(power,)             746
(injury,)            700
(landed,)            672
(airplane,)          666
(main,)              629
(reported,)          623
(control,)           606
(passenger,)         573
(accident,)          569
(pic,)               560
(back,)              530
(approach,)          527
(field,)             523
(departed,)          522
(wind,)              520
(student,)           493
(propeller,)         490
(side,)              486
(takeoff,)           480
(condition,)         463


In [31]:
(pd.Series(nltk.ngrams(words_sdr, 1)).value_counts())[:60]

(iaw,)            30097
(per,)            26373
(srm,)            22529
(amm,)            22193
(replaced,)       21455
(door,)           16320
(found,)          15841
(check,)          13497
(aft,)            13286
(removed,)        12876
(light,)          12685
(corrosion,)      12492
(aircraft,)       12425
(floor,)          11960
(b,)              11842
(1,)              11237
(repair,)         10947
(2,)              10356
(emergency,)      10315
(maintenance,)     9298
(panel,)           9168
(ops,)             9041
(cargo,)           8210
(cracked,)         8179
(ref,)             7783
(good,)            7756
(grounded,)        7713
(rr,)              7449
(repaired,)        7306
(inspection,)      7233
(wing,)            7026
(corroded,)        6983
(sta,)             6902
(cabin,)           6864
(performed,)       6844
(fwd,)             6706
(seat,)            6674
(support,)         6542
(test,)            6451
(engine,)          6323
(lower,)           6264
(rh,)           

In [32]:
(pd.Series(nltk.ngrams(words_eon, 1)).value_counts())[:60]

(reported,)       4788
(landed,)         3900
(aircraft,)       3496
(without,)        3469
(incident,)       3250
(due,)            2747
(emergency,)      2526
(runway,)         2399
(taxied,)         1689
(issue,)          1540
(medical,)        1533
(takeoff,)        1494
(passenger,)      1384
(eviation,)       1339
(declared,)       1270
(aborted,)        1267
(pilot,)          1188
(foot,)           1130
(gate,)           1110
(returned,)       1039
(rwy,)            1022
(ramp,)           1003
(notified,)        921
(engine,)          847
(acft,)            837
(damage,)          812
(loss,)            801
(landing,)         766
(departed,)        718
(gear,)            706
(separation,)      704
(atc,)             658
(back,)            658
(bird,)            647
(flight,)          640
(injury,)          624
(reporting,)       615
(c172,)            611
(laser,)           598
(departure,)       593
(pob,)             585
(mile,)            566
(b738,)            536
(diverted,)